In [169]:
import pandas as pd
import numpy as np
import requests
import json
import hvplot.pandas
from dotenv import load_dotenv
from datetime import date
import os
from scipy import stats
from numpy.lib.stride_tricks import as_strided
from numpy.lib import pad
import pad
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
load_dotenv()

True

In [212]:
print(pd.__version__)

1.0.3


In [3]:
fmp_api = os.getenv('FMP_API')

In [ ]:
as_strided()

In [887]:
class TechnicalIndicators:
    
    def __init__(self, stock_list):
        
#         self.ticker = ticker
        self.stock_list = stock_list
    
    def rolling_spearman(self, seqa, seqb, window):
        stridea = seqa.values.strides[0]
        ssa = as_strided(seqa, shape=[len(seqa) - window + 1, window], strides=[stridea, stridea])
        strideb = seqa.values.strides[0]
        ssb = as_strided(seqb, shape=[len(seqb) - window + 1, window], strides =[strideb, strideb])
        ar = pd.DataFrame(ssa)
        br = pd.DataFrame(ssb)
        ar = ar.rank(1)
        br = br.rank(1)
        corrs = ar.corrwith(br, 1)
        return pad(corrs, (window - 1, 0), 'constant', constant_values=np.nan)
    
    def create_price_df(self,ticker, period='daily'):
        
        if period =='hourly':
            api_url = 'https://fmpcloud.io/api/v3/historical-chart/1hour'
        else:
            api_url = 'https://fmpcloud.io/api/v3/historical-price-full'
            
        ticker_df = json.loads(requests.get(f"{api_url}/{ticker}?apikey={fmp_api}").content)['historical']
        data = pd.DataFrame(ticker_df).set_index('date')[::-1]
        data['Date'] = data.index
        data.index = data.index.astype('datetime64[ns]')
        
        return data
    
    def use_csvs(self, ticker):
    
        data = pd.read_csv("../FilesExportIndividualStockDFs_Big/"+ticker+"_combined_df.csv", index_col='Date', parse_dates=True)
        
        return data
        
        
        
    def bollinger_bands(self, dataframe,period=20):
        data = dataframe.copy()
        data['middle_band'] = data[['adjClose']].rolling(window=period).mean()
        data[str(period)+'_day_stdev'] = data[['adjClose']].rolling(window=period).std()
        data['upper_band'] = data['middle_band']+2*data[str(period)+'_day_stdev']
        data['lower_band'] = data['middle_band'] - 2*data[str(period)+'_day_stdev']
        data['spread'] = data['upper_band'] + data['lower_band']
        data['change_in_spread'] = data['spread']/data['spread'].shift(1)-1
        data[str(period)+"_return"] = data['adjClose']/data['adjClose'].shift(period)-1
        data['bollinger_signal'] = data['change_in_spread'].rank(ascending=False, pct=True)
        data.dropna()
        
        return data
    
    def dema(self, dataframe, period1=10, period2=20):
        data = dataframe.copy()
        data[str(period1)+'ema1'] = dataframe[['adjClose']].ewm(span=period1, adjust=False).mean()
        data[str(period1)+'ema2'] = data[str(period1)+'ema1'].ewm(span=period1, adjust=False).mean()
        data[str(period1)+'dema'] = 2*data[str(period1)+'ema1'] - data[str(period1)+'ema2']
        data[str(period2)+'ema1'] = data[['adjClose']].ewm(span=period2, adjust=False).mean()
        data[str(period2)+'ema2'] = data[str(period2)+'ema1'].ewm(span=period2, adjust=False).mean()
        data[str(period2)+'dema'] = 2*data[str(period2)+'ema1'] - data[str(period2)+'ema2']
        data[str(period1)+"_return"] = data['adjClose']/data['adjClose'].shift(period1)-1
        data['spread'] = data[str(period1)+'dema'] - data[str(period2)+'dema']
        data['dema_signal'] = data['spread'].rank(ascending=True, pct=True)
        data = data.dropna()
        return data
    
    def price_momentum(self, dataframe, smoothing1=0.0571, smoothing2=0.1, periods1=15, periods2=10):
        data = dataframe.copy()
        data['smoothing_factor'] = smoothing1
        data[str(periods1)+"average"] = data['changeOverTime'].rolling(window=periods1).mean()
        smoothing_factor_list = [data.iloc[periods1][str(periods1)+"average"]]
        data = data.dropna()
        i=1
        j=0
        while i < len(data[str(periods1)+"average"]):
            smoothing_factor = data.iloc[i]['changeOverTime']*data.iloc[i]['smoothing_factor'] + smoothing_factor_list[j]*(1-data.iloc[i]['smoothing_factor'])
            smoothing_factor_list.append(smoothing_factor)
            j+=1
            i+=1
        data['35d_custom_smoothing'] = smoothing_factor_list
        data['35d_custom_10'] = data['35d_custom_smoothing']*10
        data['smoothing_factor2'] = smoothing2
        data[str(periods2)+"average"] = data['35d_custom_10'].rolling(window=periods2).mean()
        data = data.dropna()
        smoothing_factor_list2 = [data.iloc[0][str(periods2)+"average"]]
        i=1
        j=0
        while i < len(data[str(periods2)+"average"]):
            smoothing_factor = (data.iloc[i]['35d_custom_10'] - smoothing_factor_list2[j])*data.iloc[i]['smoothing_factor2'] + smoothing_factor_list2[j]
            smoothing_factor_list2.append(smoothing_factor)
            j+=1
            i+=1
        data[str(periods2)+'d_custom_smoothing'] = smoothing_factor_list2
        data[str(periods2)+"_return"] = data['adjClose']/data['adjClose'].shift(periods2)-1
#         data['signal'] = np.where(data[str(periods2)+'d_custom_smoothing'] > data[str(periods2)+'d_custom_smoothing'].shift(1), 1.0, 0.0)
#         data = data.rename(columns={'signal':'price_mo'})
        return data
    
    def get_ichimoku_cloud(self, dataframe, period1=4, period2=8, period3=15):
        
        #TODO generate signal, ichimoku works better in current market regime with shorter periods, being able to respond faster to events than a traditional version
        # The conversion crossing the base would be the signal
        
        data = dataframe.copy()
        data['conversion_line'] = data[['adjClose']].rolling(window=period1).mean()
        data['base_line'] = data[['adjClose']].rolling(window=period2).mean()
        data['senkou_spanA_line'] = (data['conversion_line']+data['base_line'])/2
        data['senkou_spanB_line'] = data[['adjClose']].rolling(window=period3).mean()
        data['lagging_span'] = data['adjClose'].shift(period2)
        data = data.dropna()
        
        return data
    
    def accumulation_distribution_line(self, dataframe):
        
        ##TODO define periodicity and pass as arguments, use the mean as the signal generator, -1 is buy and and 1 is sell
        ##TODO need to add ability to ignore a -1 during a range of 1s
        
        data = dataframe.copy()
        data['money_flow_mult'] = round(((data['adjClose'] - data['low']) - (data['high'] - data['adjClose']))/(data['high'] - data['low']),2)
        data['money_flow_volume'] = data['money_flow_mult']*data['volume']
        adl = [data.iloc[0]['money_flow_volume']]
        i = 1
        while i < len(data):
            a_d_indicator = adl[i-1]+data.iloc[i]['money_flow_volume']
            adl.append(a_d_indicator)
            i+=1
        data['adl'] = adl
        data['adl_change'] = data['adl']/data['adl'].shift(1)-1
        negative_change_count = [0]*9
        i = 0
        counter = 0
        while i < len(data)-9:
            j=0
            while j < 9:
                if data.iloc[j+i]['adl_change'] <0:
                    counter+=1
                if j %19 == 0:
                    negative_change_count.append(counter)
                    counter = 0
                j+=1
            i+=1
        data['negative_change_counter'] = negative_change_count
        data['9_day_return'] = data['adjClose']/data['adjClose'].shift(9)-1
        data['adl_signal'] = self.rolling_spearman(data['adl'], data['9_day_return'], 9)

        return data
    
    def rsi(self, dataframe, periods=14):
        data = dataframe.copy()
        data['gains'] = np.where(data['changeOverTime']>0, data['changeOverTime'], 0)
        data['losses'] = np.where(data['changeOverTime']<0, np.absolute(data['changeOverTime']), 0)
        data['average_gain'] = data['gains'].rolling(window=periods).mean()
        data['average_loss'] = data['losses'].rolling(window=periods).mean()
        data['rs'] = data['average_gain']/data['average_loss']
        data['rsi'] = (100 - 100/(1+data['rs']))
        data['rsi_signal'] = data['rsi'].rank(ascending=True, pct=True)
        
        return data
    
    
    def get_ratings(self):
        
        ratings = json.loads(requests.get(f"https://fmpcloud.io/api/v3/historical-rating/{self.ticker}?limit=100&apikey={fmp_api}").content)
        ratings_df = pd.DataFrame(ratings)
        ratings_df['average_rating'] = (ratings_df['ratingScore']+ratings_df['ratingDetailsDCFScore']+ratings_df['ratingDetailsROEScore']+ratings_df['ratingDetailsROAScore'] \
                                        +ratings_df['ratingDetailsDEScore'] + ratings_df['ratingDetailsPEScore']+ ratings_df['ratingDetailsPBScore'])/7
        
        return ratings_df
        
    def get_stock_market_performances(self, dataframe):
        limit = len(dataframe)
        data = json.loads(requests.get(f"https://fmpcloud.io/api/v3/historical-sectors-performance?limit=525&apikey={fmp_api}").content)
        sector_df = pd.DataFrame(data).set_index('date')
        sector_df_clean = sector_df[::-1]
        s_p500 = json.loads(requests.get(f"https://fmpcloud.io/api/v3/historical-price-full/^SP500TR?from="+sector_df_clean.index[0]+"&to="+sector_df_clean.index[-1]+"&apikey="+fmp_api).content)['historical']
        sp_df = pd.DataFrame(s_p500)
        sp_df_clean = sp_df[::-1]
        
        return sp_df_clean

    def get_all_indicators(self, ticker):
        dataframe = self.create_price_df(ticker)
        bb = self.bollinger_bands(dataframe)
        dema = self.dema(dataframe)
        adl = self.accumulation_distribution_line(dataframe)
        rsi = self.rsi(dataframe)
        
        dataframe['bollinger_signal'] = bb['bollinger_signal']
        dataframe['dema_signal'] = dema['dema_signal']
        dataframe['adl_signal'] = adl['adl_signal']
        dataframe['rsi_signal'] = rsi['rsi_signal']
        return dataframe
    
    def merge_data(self):
        ticker_list = self.stock_list[0:50]
        for stock in ticker_list:
            stock_csv = self.use_csvs(stock)
            stock_csv['Date'] = stock_csv.index.astype("string")
            stock_csv = stock_csv.iloc[:-1]
            stock_csv.index.names =[""]
            stock_indicators = self.get_all_indicators(stock)
            stock_signals = stock_indicators[['Date','bollinger_signal','dema_signal', 'adl_signal', 'rsi_signal']]
            stock_signals.loc[:,'Date'] = stock_signals['Date'].astype('string')
            stock_signals.index.names =[""]
            merged_data = pd.merge(left = stock_csv, right=stock_signals, on=['Date']).set_index('Date')
            merged_data = merged_data.dropna()
            writer = pd.ExcelWriter("NewCsvs/"+stock+"_all_indicator_dfs.xlsx", engine="xlsxwriter")

            merged_data.to_excel(writer, sheet_name=stock+"_data")

            writer.save()
        
        return print("All files successfully saved")
    
    
        
      


In [888]:
stock_database = pd.read_csv('../Resources/all_symbol_list.csv').drop('Unnamed: 0', axis=1)

In [889]:
stock_list = list(stock_database['0'])

In [890]:
stock_tehcnicals= TechnicalIndicators(stock_list)

In [891]:
stock_tehcnicals.merge_data()

C:\Users\franc\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


All files successfully saved


In [785]:
data['Date'] = data.index.astype("string")

In [786]:
data = data.iloc[:-1]

In [787]:
data.index.names =[""]

In [788]:
data.tail()

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,changeOverTime,QUANTITY_FAILS,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,Date
,,,,,,,,,,,,,,,,,,,
2021-10-25,50.110000,52.125000,49.350000,51.058500,51.058500,11106805.0,11106805.0,0.9485,1.893,50.84450,0.01893,0.0,708932.0,7975.0,3442249.0,442477.0,1711.0,914058.0,2021-10-25
2021-10-26,50.799999,51.265999,49.259998,49.549999,49.442024,6153100.0,6153100.0,-1.2500,-2.461,50.02533,-0.02461,0.0,650091.0,9263.0,2030915.0,217901.0,435.0,501646.0,2021-10-26
2021-10-27,47.480000,47.919998,45.450001,45.889999,45.789997,12864700.0,12864700.0,-1.5900,-3.349,46.42000,-0.03349,6130.0,1516833.0,12300.0,4287321.0,605246.0,631.0,1058567.0,2021-10-27
2021-10-28,47.170000,47.480000,45.510000,46.450000,46.450000,10265029.0,10265029.0,-0.7200,-1.526,46.48000,-0.01526,0.0,589780.0,10096.0,2848396.0,318833.0,1861.0,812692.0,2021-10-28
2021-10-29,46.080000,47.210000,45.380000,45.950000,45.950000,6675965.0,6675965.0,-0.1300,-0.282,46.18000,-0.00282,4.0,731877.0,2254.0,2025742.0,250745.0,1.0,515379.0,2021-10-29


In [754]:
first_stock_tehcnicals_df.head()

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,Date,bollinger_signal,dema_signal,adl_signal,rsi_signal
date,,,,,,,,,,,,,,,,,
2016-12-05,44.209999,44.689999,44.209999,44.529999,42.709209,2495000.0,2495000.0,0.32,0.724,44.47667,"December 05, 16",0.00724,2016-12-05,NaN,NaN,NaN,NaN
2016-12-06,44.580002,44.900002,44.200001,44.840000,43.006535,1136700.0,1136700.0,0.26,0.583,44.64667,"December 06, 16",0.00583,2016-12-06,NaN,NaN,NaN,NaN
2016-12-07,44.560001,44.990002,44.110001,44.990002,43.150402,1815200.0,1815200.0,0.43,0.965,44.69667,"December 07, 16",0.00965,2016-12-07,NaN,NaN,NaN,NaN
2016-12-08,44.990002,45.830002,44.770000,45.799999,43.927280,1848400.0,1848400.0,0.81,1.800,45.46667,"December 08, 16",0.01800,2016-12-08,NaN,NaN,NaN,NaN
2016-12-09,45.910000,46.320000,45.849998,46.299999,44.406837,1937500.0,1937500.0,0.39,0.849,46.15667,"December 09, 16",0.00849,2016-12-09,NaN,NaN,NaN,NaN


In [874]:
ready_for_merge = first_stock_tehcnicals_df[['Date', 'bollinger_signal','dema_signal', 'adl_signal', 'rsi_signal']]

In [875]:
ready_for_merge.loc[:,'Date'] = ready_for_merge['Date'].astype('string')

In [868]:
ready_for_merge.index.names =[""]

In [871]:
ready_for_merge

,Date,bollinger_signal,dema_signal,adl_signal,rsi_signal
,,,,,
2016-12-05,2016-12-05,NaN,NaN,NaN,NaN
2016-12-06,2016-12-06,NaN,NaN,NaN,NaN
2016-12-07,2016-12-07,NaN,NaN,NaN,NaN
2016-12-08,2016-12-08,NaN,NaN,NaN,NaN
2016-12-09,2016-12-09,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2021-11-29,2021-11-29,0.780468,0.052423,0.833333,0.212681
2021-11-30,2021-11-30,0.845843,0.030183,0.833333,0.173355
2021-12-01,2021-12-01,0.908797,0.016680,0.866667,0.106742


In [796]:
merged_data = pd.merge(left = data, right=ready_for_merge, on=['Date']).set_index('Date')

In [797]:
merged_data

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,...,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,bollinger_signal,dema_signal,adl_signal,rsi_signal
Date,,,,,,,,,,,,,,,,,,,,,
2016-12-05,29.030001,31.277000,28.889999,31.219999,31.151966,7034100.0,7034100.0,2.1900,7.544,30.46233,...,1076906.0,24846.0,2484490.0,93735.0,0.0,278835.0,NaN,NaN,NaN,NaN
2016-12-06,30.709999,31.190001,30.209999,31.150000,31.082119,3248600.0,3248600.0,0.4400,1.433,30.85000,...,402003.0,1109.0,743802.0,38041.0,144.0,69899.0,NaN,NaN,NaN,NaN
2016-12-07,31.299999,31.889999,30.764999,30.900000,30.832664,5778300.0,5778300.0,-0.4000,-1.278,31.18500,...,474924.0,2563.0,1492973.0,26345.0,0.0,103799.0,NaN,NaN,NaN,NaN
2016-12-08,31.280001,31.840000,30.799999,31.309999,31.241770,3064500.0,3064500.0,0.0300,0.096,31.31667,...,452123.0,4119.0,879645.0,35032.0,0.0,98776.0,NaN,NaN,NaN,NaN
2016-12-09,31.309999,32.110001,31.040001,32.049999,31.980158,4344300.0,4344300.0,0.7400,2.363,31.73333,...,662871.0,5978.0,1646578.0,16489.0,24.0,58999.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-25,50.110000,52.125000,49.350000,51.058500,51.058500,11106805.0,11106805.0,0.9485,1.893,50.84450,...,708932.0,7975.0,3442249.0,442477.0,1711.0,914058.0,0.880549,0.987292,0.633333,0.296148
2021-10-26,50.799999,51.265999,49.259998,49.549999,49.442024,6153100.0,6153100.0,-1.2500,-2.461,50.02533,...,650091.0,9263.0,2030915.0,217901.0,435.0,501646.0,0.718321,0.988880,0.733333,0.170947
2021-10-27,47.480000,47.919998,45.450001,45.889999,45.789997,12864700.0,12864700.0,-1.5900,-3.349,46.42000,...,1516833.0,12300.0,4287321.0,605246.0,631.0,1058567.0,0.800646,0.981732,0.800000,0.085875
